### 끝에 붙어있는 " .str " 가 무엇이고, pd.melt의 작동원리는 무엇일까 ??

In [32]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [58]:
df = pd.read_csv('insuline_test.csv')
df

,이름,경구용_인슐린,주사제_인슐린,혈당수치_시작,혈당수치_끝,혈당수치_변화
0,권지경,41u - 48u,-,7.63,7.20,NaN
1,정승운,-,40u - 45u,7.56,7.09,0.97
2,김미수,-,39u - 36u,7.68,7.25,NaN
3,김혀석,33u - 36u,-,7.97,7.62,0.35
4,강서윤,-,33u - 29u,7.78,7.46,0.32
...,...,...,...,...,...,...
345,황청운,32u - 37u,-,7.75,7.41,0.34
346,최정재,-,28u - 26u,7.96,7.51,0.95
347,이수예,48u - 56u,-,7.74,7.44,0.30
348,지이하,-,42u - 44u,7.68,7.21,NaN


In [59]:
df.혈당수치_변화.isnull().sum()

# EDA 진행. 혈당수치 시작 - 끝

137

In [60]:
df.혈당수치_변화 = (df.혈당수치_시작 - df.혈당수치_끝)
df
# 한 컬럼에 두가지 feature가 들어있으므로, 분리시켜주는 것이 맞겠다 
# melt를 써보자.

,이름,경구용_인슐린,주사제_인슐린,혈당수치_시작,혈당수치_끝,혈당수치_변화
0,권지경,41u - 48u,-,7.63,7.20,0.43
1,정승운,-,40u - 45u,7.56,7.09,0.47
2,김미수,-,39u - 36u,7.68,7.25,0.43
3,김혀석,33u - 36u,-,7.97,7.62,0.35
4,강서윤,-,33u - 29u,7.78,7.46,0.32
...,...,...,...,...,...,...
345,황청운,32u - 37u,-,7.75,7.41,0.34
346,최정재,-,28u - 26u,7.96,7.51,0.45
347,이수예,48u - 56u,-,7.74,7.44,0.30
348,지이하,-,42u - 44u,7.68,7.21,0.47


In [61]:
df2 = pd.melt(df, id_vars = ['이름', '혈당수치_시작', '혈당수치_끝', '혈당수치_변화'],
                  var_name = '인슐린_종류', value_name = '투여량') 

# id_vars 변하지 않는 고정시켜놓을 컬럼들.
# var_name id vars에 없는 컬럼의 이름 (경구용_인슐린, 주사제_인슐린) 을 인슐린_종류의 값으로 넣게됨. 
# value_name 경구용_인슐린, 주사제_인슐린의 값들을 투여량이라는 컬럼을 새로 만들어 내용을 채우게 됨.

# 그렇게되면 경구용_인슐린, 주사제_인슐린 두 값을 표현하기 위해서 컬럼의 길이가 두배가 된다.
# 예를들어 0번 로우의 권지경은 경구용 인슐린만 맞았지만 주사제 인슐린도 나타내게되어 
# 한개의 권지경 row가 더 생성되고 인슐린_종류에는 주사제_인슐린, 값으로는 -가 표현된다.

df2.query("이름 == '권지경'")

,이름,혈당수치_시작,혈당수치_끝,혈당수치_변화,인슐린_종류,투여량
0,권지경,7.63,7.2,0.43,경구용_인슐린,41u - 48u
350,권지경,7.63,7.2,0.43,주사제_인슐린,-


In [62]:
df2

,이름,혈당수치_시작,혈당수치_끝,혈당수치_변화,인슐린_종류,투여량
0,권지경,7.63,7.20,0.43,경구용_인슐린,41u - 48u
1,정승운,7.56,7.09,0.47,경구용_인슐린,-
2,김미수,7.68,7.25,0.43,경구용_인슐린,-
3,김혀석,7.97,7.62,0.35,경구용_인슐린,33u - 36u
4,강서윤,7.78,7.46,0.32,경구용_인슐린,-
...,...,...,...,...,...,...
695,황청운,7.75,7.41,0.34,주사제_인슐린,-
696,최정재,7.96,7.51,0.45,주사제_인슐린,28u - 26u
697,이수예,7.74,7.44,0.30,주사제_인슐린,-
698,지이하,7.68,7.21,0.47,주사제_인슐린,42u - 44u


#### 권지경의 case로 rows가 왜 2배가 된 것인지 알 수있다.

그렇다면 -값만 지우게 된다면, 경구용을 맞은사람은 경구용만 맞은 것으로 표현할 수 있기 때문에 

다시 350개로 줄어들고 보기에 간편하고 깔끔하게 만들 수 있을 것이다.

In [79]:
df2 = df2[df2.투여량 != "-"]

df2

,이름,혈당수치_시작,혈당수치_끝,혈당수치_변화,인슐린_종류,투여량,시작_투여량,마지막_투여량
0,권지경,7.63,7.20,0.43,경구용_인슐린,41u - 48u,0,1
3,김혀석,7.97,7.62,0.35,경구용_인슐린,33u - 36u,0,1
6,김지연,7.65,7.27,0.38,경구용_인슐린,37u - 42u,0,1
7,윤초영,7.89,7.55,0.34,경구용_인슐린,31u - 38u,0,1
9,강서연,7.76,7.37,0.39,경구용_인슐린,30u - 36u,0,1
...,...,...,...,...,...,...,...,...
688,박채욱,7.51,7.06,0.45,주사제_인슐린,55u - 51u,0,1
690,김승연,7.67,7.30,0.37,주사제_인슐린,26u - 23u,0,1
694,안재주,9.21,8.80,0.41,주사제_인슐린,22u - 23u,0,1
696,최정재,7.96,7.51,0.45,주사제_인슐린,28u - 26u,0,1


이젠 2개의 feature로 구성된 투여량 컬럼의 값들을 

시작 투여량과 마지막 투여량으로 변경한 후

한 개의 값만을 할당시켜야 할 것.

In [64]:
df2['시작_투여량'], df2['마지막_투여량'] = df2['투여량'].str.split(' - ', 1).str
                                                       #여기 있는 str은 투여량이라는 컬럼 전체에 split을 적용하겠다 라는 뜻
df2

,이름,혈당수치_시작,혈당수치_끝,혈당수치_변화,인슐린_종류,투여량,시작_투여량,마지막_투여량
0,권지경,7.63,7.20,0.43,경구용_인슐린,41u - 48u,41u,48u
3,김혀석,7.97,7.62,0.35,경구용_인슐린,33u - 36u,33u,36u
6,김지연,7.65,7.27,0.38,경구용_인슐린,37u - 42u,37u,42u
7,윤초영,7.89,7.55,0.34,경구용_인슐린,31u - 38u,31u,38u
9,강서연,7.76,7.37,0.39,경구용_인슐린,30u - 36u,30u,36u
...,...,...,...,...,...,...,...,...
688,박채욱,7.51,7.06,0.45,주사제_인슐린,55u - 51u,55u,51u
690,김승연,7.67,7.30,0.37,주사제_인슐린,26u - 23u,26u,23u
694,안재주,9.21,8.80,0.41,주사제_인슐린,22u - 23u,22u,23u
696,최정재,7.96,7.51,0.45,주사제_인슐린,28u - 26u,28u,26u


Q. 맨 마지막에 있는 .str의 역할은 무엇인가요??
A. 벡터라이징 하겠다는 뜻 입니다.

In [67]:
# 벡터라이징이 무엇인고하면,

v1 = ['a','b','c','d','e','f'] # 라는 리스트가 있다고 했을 때 

v1_1 , v1_2 = [['a','b','c'],['d','e','f']] # 이렇게 각각을 벡터로 표현하는 것입니다.

v1_1 , v1_2 # 이런 식으로 두 개의 변수에 나눠담게 됩니다.

(['a', 'b', 'c'], ['d', 'e', 'f'])

In [69]:
np.ndim(v1_1), np.ndim(v1_2) #각각 벡터의 차원을 나타내는 메소드.

(1, 1)

그래서 요약을 하자면 

예를 들어 첫 번째 투여량에 있는 41u - 48u 이 값으로 설명을 한다면, 

-를 뗀 후 1개씩 나누어 '시작_투여량',과 '마지막_투여량'에 각각 담는다라는 뜻 입니다.

만약 끝에 .str을 안 붙이게 된다면 어떻게 될까요?

In [74]:
df2['시작_투여량'], df2['마지막_투여량'] = df2['투여량'].str.split(' - ', 1)

ValueError: too many values to unpack (expected 2)

이런 식으로  too many values to unpack (expected 2) 오류가 발생하게 됩니다.

왜냐하면, 투여량에 있는 값을 나누어서 각각의 값을 df2['시작_투여량'], df2['마지막_투여량']에 분배하는 것인데,

split한 값을 벡터라이징 하지 않았기 때문에 각각에 할당할수 없다고 뜨는 오류인 것으로 해석할 수 있겠습니다.

또 다른 문제로는~ 

In [71]:
k, j = map(int, input().split()) # 이런 식에서 2개의 숫자만을 입력해야하는데 3개를 입력해버린다면,
                                 # input을 2 3 4를 넣게되면

ValueError: too many values to unpack (expected 2)

정의한 변수는 2개 뿐인데, 3개의 인풋값을 넣으려고하니 발생한 오류입니다.

이 예제들을 통하여 " too many values to unpack (expected 2) 오류가 언제 발생하겠구나 ! " 라는 것을 알 수 있습니다.

In [82]:
# df2['시작_투여량'], df2['마지막_투여량'] = df2['투여량'].str.split(' - ', 1).str

# 이 코드의 경우 더 간단하게 할 수 있는 방법으로는 

df2[['시작_투여량','마지막_투여량']] = df2['투여량'].str.split(' - ', n = 1, expand = True)
# Expand the split strings into separate columns. => 공식 문서에서의 expand = True의 설명입니다.



# 이런 식으로 같은 결과값을 얻을 수 있습니다.
df2


,이름,혈당수치_시작,혈당수치_끝,혈당수치_변화,인슐린_종류,투여량,시작_투여량,마지막_투여량
0,권지경,7.63,7.20,0.43,경구용_인슐린,41u - 48u,41u,48u
3,김혀석,7.97,7.62,0.35,경구용_인슐린,33u - 36u,33u,36u
6,김지연,7.65,7.27,0.38,경구용_인슐린,37u - 42u,37u,42u
7,윤초영,7.89,7.55,0.34,경구용_인슐린,31u - 38u,31u,38u
9,강서연,7.76,7.37,0.39,경구용_인슐린,30u - 36u,30u,36u
...,...,...,...,...,...,...,...,...
688,박채욱,7.51,7.06,0.45,주사제_인슐린,55u - 51u,55u,51u
690,김승연,7.67,7.30,0.37,주사제_인슐린,26u - 23u,26u,23u
694,안재주,9.21,8.80,0.41,주사제_인슐린,22u - 23u,22u,23u
696,최정재,7.96,7.51,0.45,주사제_인슐린,28u - 26u,28u,26u


혹시라도 틀린 부분이 있다면 말씀해주세요. 

저는 아직 부족합니다.

감사합니다.